<a href="https://colab.research.google.com/github/joynaomi81/Sentiment-Analysis-Project/blob/main/Movie_Reviews_Sentiment_Analysis_in_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [6]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv')

In [7]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [9]:
data.columns

Index(['review', 'sentiment'], dtype='object')

In [10]:
data.isnull().any()

review       False
sentiment    False
dtype: bool

In [11]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [12]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [13]:
data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [14]:
data.shape

(50000, 2)

Test normalization

Tokenization

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt



In [16]:
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

In [17]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import wordcloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

In [18]:
import spacy
import re, string, unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn. svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn. metrics import classification_report,confusion_matrix,accuracy_score
from bs4 import BeautifulSoup

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
#Tokenization of text
tokenizers=ToktokTokenizer()
#Setting English stopwords
stopsword=nltk.corpus.stopwords.words('english')

In [21]:
#Removing the noisy text
def noiseremoval_text(text):
  soup = BeautifulSoup(text, 'html.parser')
  text = soup.get_text()
  text = re.sub('\[[^]]*\]','',text)
  return text

In [22]:
#Apply function on review column
data['review']=data['review'].apply(noiseremoval_text)

In [23]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Stemming

In [24]:
#Stemming the text
def stemmer(text):
  ps=nltk.PorterStemmer()
  text= ''.join([ps.stem(word)for word in text.split()])
  return text

In [25]:
#Apply function on review column
data['review']=data['review'].apply(stemmer)

In [26]:
data.head()

,review,sentiment
0,oneoftheotherreviewhamentionthatafterwatchjust...,positive
1,awonderlittlproduction.thefilmtechniquisveriun...,positive
2,ithoughtthiwaawonderwaytospendtimeonatoohotsum...,positive
3,basicthere'afamiliwherealittlboy(jake)thinkthe...,negative
4,"pettermattei'""loveinthetimeofmoney""isavisualst...",positive


Removing stop words

In [27]:
def removing_stopwords(text, is_lower_case=False):
    tokenizers=ToktokTokenizer()
    tokens = tokenizers.tokenize(text)
    tokens = [i.strip() for i in tokens]
    if is_lower_case:
        filtokens = [i for i in tokens if i not in stop_words]
    else:
         filtokens = [i for i in tokens if i.lower() not in stop_words]
    filtered_text = ' '.join(filtokens)
    return filtered_text
    data['text'] = data['text'].apply(removing_stopwords)
      
  

In [28]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

stop_words = stopwords.words("english")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
#Apply function on column
data['review']=data['review'].apply(removing_stopwords)

In [30]:
data.head()

,review,sentiment
0,oneoftheotherreviewhamentionthatafterwatchjust...,positive
1,awonderlittlproduction.thefilmtechniquisveriun...,positive
2,ithoughtthiwaawonderwaytospendtimeonatoohotsum...,positive
3,basicthere ' afamiliwherealittlboy( jake ) thi...,negative
4,"pettermattei ' "" loveinthetimeofmoney "" isavis...",positive


# Train test split

In [31]:
#test dataset
train_reviews_data=data.review[:30000]

In [32]:
test_reviews_data=data.review[30000:]

In [33]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=2,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train=cv.fit_transform(train_reviews_data)
cv_test=cv.transform(test_reviews_data)

print('BOW_cv_train:',cv_train.shape)
print('BOW_cv_test:',cv_test.shape)


BOW_cv_train: (30000, 2778550)
BOW_cv_test: (20000, 2778550)


## **TF_IDF**

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(min_df=0,max_df=2,binary=False,ngram_range=(1,3))
tf_train=tf.fit_transform(train_reviews_data)
tf_test=tf.transform(test_reviews_data)
print('BOW_Tfid_train:',tf_train.shape)
print('BOW_Tfid_test:',tf_test.shape)

BOW_Tfid_train: (30000, 2778550)
BOW_Tfid_test: (20000, 2778550)


## **Lable encoding**

In [36]:
#labeling the sentiment data
label=LabelBinarizer()
#transformed sentiment
sentiment_data=label.fit_transform(data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [37]:
train_data=data.sentiment[:30000]

In [38]:
test_data=data.sentiment[30000:]

In [39]:
sentiment_data

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [40]:
train_data=data.sentiment[:30000]

In [41]:
test_data=data.sentiment[30000:]

In [42]:
#training the model
logistic=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#fitting the model for bag of words
lr_bow = logistic.fit(cv_train, train_data)
print(lr_bow)
#fitting the model for tfidf features
lr_tfidf=logistic.fit(tf_train,train_data)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


In [43]:
#Predicting the model for bag of words
lr_bow_predict=logistic.predict(cv_test)
print(lr_bow_predict)

['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [44]:
bow_predict=logistic.predict(cv_test)
print(bow_predict)

['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [45]:
bow_score=accuracy_score(test_data,bow_predict)
print('lr_bow_score:',bow_score)

lr_bow_score: 0.6404


In [46]:
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [47]:
lr_bow_score=accuracy_score(test_data,lr_bow_predict)
print('lr_bow_score:',lr_bow_predict)

lr_bow_score: ['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [48]:
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['positive' 'negative' 'negative' ... 'positive' 'positive' 'negative']


In [49]:
lr_tfidf_score=accuracy_score(test_data,lr_tfidf_predict)
print('lr_tfidf_score:',lr_tfidf_score)

lr_tfidf_score: 0.6405


In [50]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv')

# Convert categorical variables into numerical variables using one-hot encoding
df = pd.get_dummies(df, columns=["review"])

# Check the data again
print(df.head(2))


  sentiment  \
0  positive   
1  positive   

   review_A Turkish Bath sequence in a film noir located in New York in the 50's, that must be a hint at something ! Something that curiously, in all the previous comments, no one has pointed out , but seems to me essential to the understanding of this movie <br /><br />the Turkish Baths sequence: a back street at night, the entrance of a sleazy sauna, and Scalise wrapped in a sheet, getting his thighs massaged. Steve, the masseur is of the young rough boxer ( Beefcake!) type , and another guy, a bodyguard? finishes dressing up. Dixon obviously hates what he sees there and gets rough right away. We know he has a reputation for roughing up suspects. Good cop but getting out of control easy. Why is it that he hates them so much ? <br /><br />Could it be that he hates himself. This part of himself he inherited from his father ? That dark side that could lead him right at the end of the sidewalk, into the gutter ? What if that dark side lur